# PREAMBLE
<script
  src="https://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"
  type="text/javascript">
</script>

In [ ]:
import numpy as np
import pandas as pd
import numpy.linalg as la
import seaborn as sns
sns.set()
sns.set_style("ticks")
from validphys.api import API
from validphys.loader import Loader
from matplotlib import pyplot as plt
from matplotlib import cm
l = Loader()

# COMPUTATION OF $\alpha_s$

Definition of the input

In [ ]:
#Fit and PDFs (used in fit)
fit="221203-ern-001"
pdf_ori="NNPDF40_nnlo_as_01180"
pdf_fit="221203-ern-001"

In [ ]:
#Inputs for central theory: alphas=0.118
inps_central = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            theoryid=200,
            use_cuts="fromfit",
            pdf=pdf_ori)

In [ ]:
#Inputs for plus theory: alphas=0.120
inps_plus = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            theoryid=207,
            use_cuts="fromfit",
            pdf=pdf_ori)

In [ ]:
#Inputs for minus theory: alphas=0.116
inps_minus = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            theoryid=201,
            use_cuts="fromfit",
            pdf=pdf_ori)

In [ ]:
#Inputs for central theory: alphas=0.118
inps_central_fit = dict(dataset_inputs={"from_": "fit"},
            fit=fit,
            theoryid=200,
            use_cuts="fromfit",
            pdf={"from_": "fit"})

In [ ]:
#Experimental covariance matrix
C = API.groups_covmat(**inps_central)

In [ ]:
#Indexes
dsindex=API.groups_index(**inps_central)

In [ ]:
datth_central = API.group_result_table_no_table(**inps_central)

In [ ]:
datth_plus = API.group_result_table_no_table(**inps_plus)

In [ ]:
datth_minus = API.group_result_table_no_table(**inps_minus)

In [ ]:
datth_central_fit = API.group_result_table_no_table(**inps_central_fit)

In [ ]:
dat_central=datth_central["data_central"]
th_central=datth_central["theory_central"]
th_replicas=datth_central.iloc[:,2:102].to_numpy()
th_central_fit=datth_central_fit["theory_central"]
th_replicas_fit=datth_central_fit.iloc[:,2:102].to_numpy()

Computation of Eqs.(3.37)-(3.38) in [arXiv:2105.05114](https://arxiv.org/pdf/2105.05114.pdf)

In [ ]:
beta_tilde = (0.002/np.sqrt(2))*np.array([1,-1])
S_tilde = beta_tilde.T@beta_tilde

In [ ]:
delta_plus  = (1/np.sqrt(2))*(datth_plus["theory_central"] - datth_central["theory_central"]).to_numpy()
delta_minus = (1/np.sqrt(2))*(datth_minus["theory_central"] - datth_central["theory_central"]).to_numpy()
beta = [delta_plus,delta_minus]
S_hat = beta_tilde.T@beta

In [ ]:
S = np.outer(delta_plus,delta_plus)+np.outer(delta_minus,delta_minus)
S = pd.DataFrame(S,index=dsindex,columns=dsindex)
S = pd.DataFrame(S.values, index=C.index, columns=C.index)

In [ ]:
invcov = la.inv(C+S)

In [ ]:
X = np.zeros_like(C.values)
for i in range(th_replicas_fit.shape[1]):
    X = X + 1/len(th_central_fit) * np.outer((th_replicas_fit[:,i]-th_central_fit),(th_replicas_fit[:,i]-th_central_fit))

Final result

In [ ]:
delta_T_tilde = S_hat@invcov@(dat_central-th_central_fit)
P_tilde = S_hat.T@invcov@X@invcov@S_hat + (S_tilde - S_hat.T@invcov@S_hat)
pred = 0.118 + delta_T_tilde
unc  = np.sqrt(P_tilde) 
print(rf"Prediction for $\alpha_s$: {pred} +/- {unc}")